In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

## ～2021年10月19日

## 2021年10月19日～2021年10月27日

# 2021年11月16日～

## 下記のような処理を行う関数を作成する

関数名, モデル1, ... , モデルn

->

関数名, モデル1, ... , モデルn, MAPE最低値

In [ ]:
inputDF = dfInit

funcNames = inputDF.index.to_list()
modelNames = inputDF.columns.to_list()

inputDF["最低値"] = math.inf

for funcName in funcNames:
    lowestInFunc = math.inf
    seriesInFunc = inputDF.loc[funcName]
    for modelName in modelNames:
        elem = seriesInFunc[modelName]
        if elem < lowestInFunc:
            lowestInFunc = elem
        inputDF.at[funcName, "最低値"] = lowestInFunc

## 下記のような処理を行う関数を作成する

{ベンチマーク名(str):MAPE表(DF)}

        
inputDict = ["benchmarkName":inputDF]

->

ベンチマーク名, MAPE平均

In [ ]:
inputDict = {"benchmarkName": inputDF}

benchmarkNames = []
lowestMAPEAverage = []

for benchmarkName in inputDict.keys():
    averageSeries = inputDict[benchmarkName].mean()
    benchmarkNames.append(benchmarkName)
    lowestMAPEAverage.append(averageSeries["最低値"])

resultDF = pd.DataFrame(
    index=benchmarkNames, data=lowestMAPEAverage, columns=["平均絶対誤差"]
)
resultDF

In [ ]:
inputDF

In [ ]:
dfBase

In [ ]:
# 引数として渡されたDFに最低MAPEの列を追加する関数
# 引数：DF[関数名, モデル名1, ... , モデル名n]
# 返値：DF[関数名, モデル名1, ... , モデル名n, 最低MAPE]


def addLowestMAPEColumn(resultDF=dfInit):

    funcNames = resultDF.index.to_list()
    modelNames = resultDF.columns.to_list()

    result["最低値"] = math.inf

    for funcName in funcNames:
        lowestInFunc = math.inf
        seriesInFunc = resultDF.loc[funcName]
        for modelName in modelNames:
            elem = seriesInFunc[modelName]
            if elem < lowestInFunc:
                lowestInFunc = elem
            result.at[funcName, "最低値"] = lowestInFunc

    return resultDF


# 引数として渡された辞書からDFを返す関数
# 引数：辞書{ベンチマーク名:DF[ベンチマーク名, 最低MAPEの平均]}
# 返値：DF[ベンチマーク名, 最低MAPEの平均]


def returnDFBenchmarkNameAndAverageLowestMAPE(inputDict={"benchmarkName": inputDF}):

    benchmarkNames = []
    lowestMAPEAverage = []

    for benchmarkName in inputDict.keys():
        averageSeries = inputDict[benchmarkName].mean()
        benchmarkNames.append(benchmarkName)
        lowestMAPEAverage.append(averageSeries["最低値"])

    resultDF = pd.DataFrame(
        index=benchmarkNames, data=lowestMAPEAverage, columns=["平均絶対誤差"]
    )
    return resultDF

In [ ]:
# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
date = "2021年11月15日"
for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    print(
        dfBase.to_latex(
            caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
            label=f"{date}BaseMAPE{benchmarkName}",
        )
    )
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    print(
        dfInit.to_latex(
            caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
            label=f"{date}InitMAPE{benchmarkName}",
        )
    )

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
date = "2021年11月11日"
for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    print(
        dfBase.to_latex(
            caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
            label=f"{date}BaseMAPE{benchmarkName}",
        )
    )
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    print(
        dfInit.to_latex(
            caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
            label=f"{date}InitMAPE{benchmarkName}",
        )
    )

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)